In [2]:
import pandas as pd 
data = pd.read_csv("news_summary.csv", encoding = 'latin1')

In [3]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math

In [4]:
import bs4 as bs  
import urllib.request  
import re
scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')  
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:  
    article_text += p.text

In [5]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
article_text = re.sub(r'\s+', ' ', article_text) 

In [6]:
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  

In [7]:
def computeTFIDF(TF_scores, IDF_scores):
    TFIDF_scores = []
    for j in IDF_scores:
        for i in TF_scores:
            if j['key'] == i['key'] and j['doc_id'] == i['doc_id']:
                temp = {'doc_id' : j['doc_id'],
                        'TFIDF_score' : j['IDF_score' ]*i[ 'TF_score' ],
                        'key' : i['key']}
        TFIDF_scores.append(temp)
    return TFIDF_scores

In [8]:
def count_words(sent):
    count = 0
    words = word_tokenize(sent)
    for word in words:
        count += 1
    return count


In [9]:
def create_freq_dict(sents):
    i = 0
    freqDict_list = []
    for sent in sents:
        i += 1
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1
            temp = {'doc_id' : i, 'freq_dict': freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

In [10]:
def remove_string_special_characters(s):
    stripped = re.sub('[^\w\s]','', s)
    stripped = re.sub('_', '', stripped)
    stripped = re.sub('\s+', ' ', stripped)
    stripped = stripped.strip()
    return stripped

In [11]:
def get_doc(sent):
    doc_info = []
    i = 0
    for sent in text_sents_clean:
        i += 1
        count = count_words(sent)
        temp = {'doc_id' : i, 'doc_length' : count}
        doc_info.append(temp)
    return doc_info

In [12]:
def computeTF(doc_info, freqDict_list):
    TF_scores = []
    for tempDict in freqDict_list:
        id = tempDict['doc_id']
        for k in tempDict['freq_dict']:
            temp = {'doc_id' : id,
                    'TF_score' : tempDict['freq_dict'][k]/ doc_info[id-1]['doc_length'],
                    'key' : k}
            TF_scores.append(temp)
    return TF_scores

In [13]:
 def computeIDF(doc_info, freqDict_list):
        IDF_scores = []
        counter = 0
        for dict in freqDict_list:
            counter += 1
            for k in dict['freq_dict'].keys():
                count= sum([k in tempDict[ 'freq_dict'] for tempDict in freqDict_list])
                temp = {'doc_id' : counter, 'IDF_score' : math.log(len(doc_info)/count),'key' : k}
                IDF_scores.append( temp)
        return IDF_scores

In [14]:
text_sents = sent_tokenize(article_text)

In [15]:
text_sents_clean = [remove_string_special_characters(s) for s in text_sents]

In [16]:
doc_info = get_doc(text_sents_clean)

In [17]:
freqDict_list = create_freq_dict(text_sents_clean)

In [18]:
IDF_scores = computeIDF(doc_info, freqDict_list)

In [19]:
TF_scores = computeTF(doc_info, freqDict_list)

In [20]:
TFIDF_scores = computeTFIDF(TF_scores, IDF_scores)

In [21]:
TFIDF_scores

[{'TFIDF_score': 0.07469792908202781, 'doc_id': 1, 'key': 'in'},
 {'TFIDF_score': 0.11410556679541302, 'doc_id': 1, 'key': 'computer'},
 {'TFIDF_score': 0.1500405374637528, 'doc_id': 1, 'key': 'science'},
 {'TFIDF_score': 0.08353134083887889, 'doc_id': 1, 'key': 'artificial'},
 {'TFIDF_score': 0.3007913576926552, 'doc_id': 1, 'key': 'intelligence'},
 {'TFIDF_score': 0.04804204767716945, 'doc_id': 1, 'key': 'ai'},
 {'TFIDF_score': 0.18894826483600202, 'doc_id': 1, 'key': 'sometimes'},
 {'TFIDF_score': 0.16228875789138875, 'doc_id': 1, 'key': 'called'},
 {'TFIDF_score': 0.10038729972084641, 'doc_id': 1, 'key': 'machine'},
 {'TFIDF_score': 0.05112061335692085, 'doc_id': 1, 'key': 'is'},
 {'TFIDF_score': 0.2156077717806153, 'doc_id': 1, 'key': 'demonstrated'},
 {'TFIDF_score': 0.157727231138968, 'doc_id': 1, 'key': 'by'},
 {'TFIDF_score': 0.11550432234044668, 'doc_id': 1, 'key': 'machines'},
 {'TFIDF_score': 0.18036582055468628, 'doc_id': 1, 'key': 'contrast'},
 {'TFIDF_score': 0.026237621

In [26]:
import statistics
def get_sent_score(TFIDF_scores, text_sents, doc_info):
    sentence_info = []
    for doc in doc_info:
        sent_score = 0
        for i in range(0,len(TFIDF_scores)):
            temp_dict = TFIDF_scores[i]
            
            if doc['doc_id'] == temp_dict['doc_id']:
                sent_score += temp_dict['TFIDF_score']
        temp = {'doc_id' : doc['doc_id'], 'sent_score':sent_score,
                'sentence':text_sents[doc['doc_id']-1]}
        sentence_info.append(temp)
        
    return (sentence_info)

In [27]:
sentence_info = get_sent_score(TFIDF_scores, text_sents, doc_info)

In [28]:
def get_summary(sentence_info):
    sum = 0
    summary = []
    array = []
    
    for temp_dict in sentence_info:
        sum += temp_dict['sent_score']
        
    avg = (sum/0.8)/len(sentence_info)
    for temp_dict in sentence_info:
        array.append(temp_dict['sent_score'])
    
    stdev = statistics.stdev(array)
    
    for sent in sentence_info:
        if (sent['sent_score']) >= avg:
            summary.append(sent['sentence'])
    summary = '\n'.join(summary)
    return (summary)

In [29]:
summary = get_summary(sentence_info)

NameError: name 'temp' is not defined